# 조사 및 기사 작성 Agent 개발

멀티 에이전트 시스템 기본 개념 및 CrewAI 프레임워크 소개

In [11]:
%pip install -q crewai crewai_tools langchain langchain_community


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
import getpass
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'
# os.environ["OPENAI_API_KEY"] = getpass.getpass()


True

# Agent 생성

- 에이전트를 정의하고 `역할(role)`, `목표(goal)` 및 `배경 스토리(backstory)`를 제공합니다.
- LLM은 롤플레잉을 할 때 더 나은 성과를 거두는 것으로 나타났습니다.

### Agent: Planner

In [5]:
planner = Agent(
    role="콘텐츠 기획자",
    goal="{topic}에 대한 흥미롭고 사실에 기반한 콘텐츠 기획",
    backstory="당신은 {topic}에 대한 블로그 기사를 기획하고 있습니다. "
                "독자가 새로운 정보를 배우고,  "
                "올바른 결정을 내리는 데 도움이 될 수 있는 자료를 수집합니다. "
                "당신의 기획 내용은 콘텐츠 작성자가 이 주제에 대한 기사를 작성하는 "
                "이 주제에 대한 기사를 작성하는 기반이 됩니다.",
    allow_delegation=False,
    verbose=True,
    llm="gpt-4o-mini"
)

### Agent: Writer

In [6]:
writer = Agent(
    role="콘텐츠 작성자",
    goal="{topic} 에 대한 통찰력 있고 사실에 기반한 "
         "의견 기사를 한글로 작성하는 것",
    backstory="당신은 {topic} 에 대한 새로운 블로그 게시글을 작성하고 있습니다. "
              "이 기사는  콘텐츠 기획자가 제공하는 개요와 관련 정보를 기반으로 합니다. "
              "당신은 콘텐츠 기획자가 제공한 개요의 주요 목표와 방향을 따릅니다. "
              "또한, 객관적이고 공정한 통찰을 제공하며, "
              "이를 콘텐츠 기획자가 제공한 정보로 뒷받침합니다. "
              "블로그 게시글에서는 의견과 객관적 사실을 명확히 구분하여 서술합니다.",
    allow_delegation=False,
    verbose=True,
    llm="gpt-4o-mini"
)

### Agent: Editor

In [7]:
editor = Agent(
    role="편집자",
    goal="주어진 기사를 조직의 글쓰기 스타일에 맞게 편집합니다.",
    backstory="당신은 콘텐츠 작성자 로부터 블로그 게시글을 전달받는 편집자입니다. "
              "당신의 목표는 블로그 게시글을 검토하여 저널리즘의 최선의 관행을 따르도록 하고, "
              "의견이나 주장을 제공할 때 균형 잡힌 관점을 유지하며, "
              "가능하면 주요 논란이 되는 주제나 의견을 피하도록 하는 것입니다.",
    allow_delegation=False,
    verbose=True,
    llm="gpt-4o-mini"
)

## Task 생성

- Task를 정의하고 `설명(description)`, `예상 출력(expected_output)` 및 `에이전트(agent)`를 제공합니다.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. {topic} 에 대한 최신 트렌드, 주요 플레이어,   "
            "주요 뉴스를 우선적으로 다룬다.\n"
        "2. 대상 독자를 파악하고, 그들의 관심사와 주요 문제점을 분석한다.\n"
        "3. 소개, 핵심 내용, 그리고 콜투액션(CTA)을 포함한 "
            "상세한 콘텐츠 개요를 작성한다.\n"
        "4. SEO 키워드와 관련 데이터 또는 출처를 포함한다."
    ),
    expected_output="콘텐츠 개요, 대상 독자 분석, "
        "SEO 키워드 및 참고 자료를 포함한 "
        "포괄적인 콘텐츠 계획 문서.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. 콘텐츠 기획을 활용하여 {topic} 에 대한 "
            "매력적인 블로그 게시글을 작성하세요.\n"
        "2. SEO 키워드를 자연스럽게 포함하세요.\n"
        "3. 섹션 및 부제목을 흥미롭게 구성하세요.\n"
        "4. 글의 구조는 흥미로운 도입부, 통찰력 있는 본문, "
            "요약하는 결론으로 구성하세요.\n"
        "5. 문법 오류를 수정하고 브랜드의 톤과 "
            "일관성을 유지하세요.\n"
    ),
    expected_output="마크다운 형식으로 작성된 완성도 높은 "
        "한글로 작성된 블로그 게시글. 각 섹션은 2~3개의 단락으로 구성되어야 합니다.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("주어진 블로그 게시물을 교정하여 "
                 "문법 오류를 수정하고 "
                 "브랜드 칼라에 맞추세요."),
    expected_output="출판 준비가 완료된, 마크다운 형식의 한글로 잘 작성된 블로그 게시물. "
                    "각 섹션은 2~3개의 단락으로 구성되어야 합니다.",
    agent=editor
)

## Crew 생성

- 에이전트 Crew를 만들고, 해당 에이전트가 수행할 작업을 전달합니다.
- **참고**: task는 순차적으로 수행됩니다.
- `verbose=2`를 사용하면 실행 로그를 볼 수 있습니다.

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit]
)

## Crew 실행

In [12]:
result = crew.kickoff(inputs={"topic": "AI 에이전트"})

# Agent: 콘텐츠 기획자
## Task: 1. AI 에이전트 에 대한 최신 트렌드, 주요 플레이어,   주요 뉴스를 우선적으로 다룬다.
2. 대상 독자를 파악하고, 그들의 관심사와 주요 문제점을 분석한다.
3. 소개, 핵심 내용, 그리고 콜투액션(CTA)을 포함한 상세한 콘텐츠 개요를 작성한다.
4. SEO 키워드와 관련 데이터 또는 출처를 포함한다.


# Agent: 콘텐츠 기획자
## Final Answer: 
### 콘텐츠 개요: AI 에이전트에 대한 최신 트렌드와 주요 개발

#### 1. 소개
- **AI 에이전트란?**: AI 에이전트의 정의와 발전 배경 소개.
- **글의 목적**: 최신 트렌드, 주요 플레이어 및 뉴스에 대한 통찰 제공.
- **독자를 위한 이점**: AI 에이전트의 이해를 통해 비즈니스 및 개인 생활에 어떻게 활용할 수 있는지 설명.

#### 2. AI 에이전트 최신 트렌드
- **지속적인 진화**: AI 에이전트의 기술이 행하는 방식의 변화 추세 설명.
- **자연어 처리 기술의 발전**: 최근의 NLP 기술이 AI 에이전트의 대화 능력에 미치는 영향.
- **특정 산업에 대한 적용**: 금융, 의료, 고객 서비스 등의 분야에서의 실질적인 사례.

#### 3. 주요 플레이어
- **테슬라**: 자율주행 기술과 AI 에이전트의 통합.
- **구글**: Google Assistant의 최신 기능 소개.
- **아마존**: Alexa의 시장점유율 확대 및 개발 로드맵.
- **마이크로소프트**: Cortana의 진화 및 기업 솔루션으로서의 AI 에이전트 활용.

#### 4. 주요 뉴스
- **2023년 주요 발표**: 각 기업에서 발표한 AI 에이전트 관련 최신 기술과 제품 소개.
- **AI 이론의 변화**: 업계의 전문가들이 논의하는 새로운 이론과 전망.
- **윤리 및 규제**: AI 에이전트 사용에 영향을 미치는 최신 윤리적 쟁점 및 법률.

#### 5. 대상 독자 분석
- **대상 독자**: 기술 산업 종사자

- 실행 결과를 노트북에 마크다운으로 표시합니다.

In [13]:
from IPython.display import Markdown
Markdown(result.raw)

```markdown
# AI 에이전트: 최신 트렌드와 주요 개발

## 1. 소개

### AI 에이전트란?
AI 에이전트는 인공지능 기반의 소프트웨어 프로그램으로, 특정 작업을 자동화하고 인간과 상호작용을 통해 유용한 정보를 제공하는 역할을 합니다. 최근 몇 년 사이, AI 에이전트는 자연어 처리(NLP) 기술의 발전과 함께 빠르게 진화해왔습니다. 이러한 발전은 사용자와의 대화의 질을 높이고, 다양한 분야에서의 적용 가능성을 확대하게 만들었습니다.

### 글의 목적
본 글에서는 AI 에이전트의 최신 트렌드와 주요 플레이어들, 그리고 관련 뉴스에 대한 통찰을 제공하고자 합니다. 독자들은 이 글을 통해 AI 에이전트의 진화 과정을 이해하고, 비즈니스 및 개인 생활에서 어떻게 활용할 수 있을지에 대한 인사이트를 얻게 될 것입니다.

### 독자를 위한 이점
AI 에이전트를 이해함으로써 독자들은 최신 기술이 자산이 될 수 있는 방법을 알아볼 수 있습니다. 비즈니스 경영자들은 고객 서비스와 운영 효율성을 높이는 데 필요한 정보를 얻고, 일반 소비자들은 AI 기술을 삶의 질 향상에 어떻게 활용할 수 있을지 탐색할 수 있습니다.

## 2. AI 에이전트 최신 트렌드

### 지속적인 진화
AI 에이전트의 기술은 계속해서 진화하고 있습니다. 특히, 머신러닝과 딥러닝의 발전은 AI 에이전트가 보다 고도화된 패턴 인식을 가능하게 하며, 사용자 요구에 더욱 정확하게 대응할 수 있도록 하고 있습니다. 이제는 단순한 명령 처리에서 벗어나, 심층적인 대화와 복잡한 문제 해결이 가능한 방향으로 나아가고 있습니다.

### 자연어 처리 기술의 발전
최근의 NLP 기술 발전은 AI 에이전트의 대화 능력을 크게 향상시켰습니다. 새로운 알고리즘과 데이터 처리 기법 덕분에 AI 에이전트는 이전보다 더욱 자연스러운 대화가 가능해졌고, 이는 사용자와의 상호작용에서 더 나은 경험을 제공합니다. 예를 들어, 새로운 AI 기반의 고객 지원 시스템은 고객의 문의에 즉각적이고 정확한 반응을 할 수 있게 되었습니다.

### 특정 산업에 대한 적용
AI 에이전트는 여러 산업 분야에서 실질적인 사례를 통해 그 가능성을 보여주고 있습니다. 금융업계에서는 AI 에이전트가 고객의 개별 재무 상담을 제공하고 있으며, 의료 분야에서는 환자 진료기록을 분석하는 데 활용되고 있습니다. 고객 서비스 분야에서는 24시간 운영되는 AI 대화형 에이전트를 통해 신속한 문제 해결이 가능한 시스템이 구축되고 있습니다.

## 3. 주요 플레이어

### 테슬라
테슬라는 자율주행 기술과 AI 에이전트의 통합에 주력하고 있습니다. 자율주행 차량은 AI 에이전트를 통해 실시간 피드백을 제공하여 운전자의 안전과 편의를 향상시키고 있습니다. 이는 교통 데이터 분석과 연결되어 차량 운영을 보다 효율적으로 만드는 데 기여하고 있습니다.

### 구글
구글의 Google Assistant는 최신 기능을 통해 사용자와 보다 자연스러운 대화를 지원하고 있습니다. 사용자의 개인화된 정보 제공 및 스마트 홈 기기와의 통합을 통해 AI 에이전트의 효용을 극대화하고 있습니다. 특히, 최근 발표된 기능에서는 사용자 맞춤형 알림과 일정 관리를 지원하여 더욱 진화된 서비스를 제공합니다.

### 아마존
아마존의 Alexa는 시장 점유율이 지속적으로 확대되고 있으며, 다양한 개발 로드맵을 통해 더욱 다채로운 서비스가 제공되고 있습니다. Alexa는 여러 스마트 기기와의 호환성을 통해 사용자의 생활을 편리하게 만들어 주며, 고객의 요구를 분석하여 더 나은 사용자 경험을 창출하고 있습니다.

### 마이크로소프트
마이크로소프트의 Cortana는 기업 솔루션으로서 AI 에이전트를 활용 가능성을 강조하고 있습니다. Cortana는 비즈니스 환경에서 일정 관리, 이메일 처리 및 회의 지원 등을 통해 사용자 생산성을 높이고 있습니다. AI 기술의 활용을 통해 기업의 효율성을 증대시키는 데 기여하고 있습니다.

## 4. 주요 뉴스

### 2023년 주요 발표
2023년에는 여러 기업에서 AI 에이전트 관련 최신 기술과 제품이 발표되었습니다. 특히, 머신러닝 효율성을 높이기 위한 새로운 프레임워크와 AI 서비스의 상호운용성 증진에 대한 자료들이 공개되며 관심을 끌고 있습니다. 이러한 발표는 AI 에이전트의 활용 범위를 더욱 넓히는 사례로 주목받고 있습니다.

### AI 이론의 변화
AI 업계의 전문가들은 AI 에이전트와 관련된 다양한 새로운 이론과 전망을 논의하고 있습니다. 사람과 기계 간의 상호작용이 더욱 깊어짐에 따라, AI의 역할과 윤리에 대한 논의도 활발해지고 있습니다. 이는 업계 전반에 걸쳐 많은 변화와 혁신을 이끌 것으로 기대됩니다.

### 윤리 및 규제
AI 에이전트의 사용이 확대됨에 따라 최신 윤리적 쟁점과 법률에 대한 논의도 더욱 중요해지고 있습니다. 데이터 프라이버시와 AI의 책임성에 대한 질문들이 제기됨에 따라, 기업들은 윤리적인 기준을 설정하고 이를 준수해야 할 필요성이 커지고 있습니다.

## 5. 대상 독자 분석

### 대상 독자
AI 에이전트를 주제로 한 이 글의 주요 독자는 기술 산업 종사자, 기업 경영자, 일반 소비자입니다. 이들은 AI 기술의 최신 동향에 대한 관심이 크며, 이를 비즈니스에서 활용할 방법에 대해 알고자 하는 열망이 있습니다.

### 관심사
대부분의 독자들은 AI 기술의 최신 트렌드와 이 기술이 비즈니스를 어떻게 변화시킬 수 있는지에 대한 정보를 찾고 있습니다. 특히, AI의 실제 활용 사례와 시장 흐름에 대한 통찰을 통해 긍정적인 결정을 내리는 데 도움이 될 것입니다.

### 문제점
하지만 기술에 대한 이해의 부족, 정보의 파편화, 윤리적 우려 등의 문제점이 여전히 존재합니다. 이러한 문제들을 해결하기 위해서는 명확하고 신뢰할 수 있는 정보 제공이 필요합니다.

## 6. 콜투액션 (CTA)

AI 에이전트의 세계에 대한 깊이 있는 정보와 최신 트렌드를 놓치지 않으려면, 뉴스레터를 구독해 주세요. 효율적인 비즈니스 및 개인적인 프로젝트에 AI 에이전트를 도입하는 방법에 대한 상담도 요청하실 수 있습니다. 여러분의 미래를 AI와 함께 만들어 보세요!
```  
이 블로그 게시물은 AI 에이전트의 최신 동향을 주제로 작성되었으며, 다양한 산업에서의 적용 사례와 최신 뉴스, 독자에게 필요할 정보를 제공하는 데 초점을 두고 있습니다. SEO 키워드를 적절하게 포함하였고, 정보 전달과 브랜드 톤을 일관성 있게 유지하였습니다.

## 직접 시도해보세요

- 원하는 주제를 제출하고 에이전트가 무엇을 내놓는지 보세요!

In [14]:
topic = "온톨로지 기반 공간 AI 서비스"
result = crew.kickoff(inputs={"topic": topic})

# Agent: 콘텐츠 기획자
## Task: 1. 온톨로지 기반 공간 AI 서비스 에 대한 최신 트렌드, 주요 플레이어,   주요 뉴스를 우선적으로 다룬다.
2. 대상 독자를 파악하고, 그들의 관심사와 주요 문제점을 분석한다.
3. 소개, 핵심 내용, 그리고 콜투액션(CTA)을 포함한 상세한 콘텐츠 개요를 작성한다.
4. SEO 키워드와 관련 데이터 또는 출처를 포함한다.


# Agent: 콘텐츠 기획자
## Final Answer: 
### 콘텐츠 개요  

**제목:** 온톨로지 기반 공간 AI 서비스의 최신 트렌드와 주요 플레이어

**소개:**  
온톨로지 기반 공간 AI 서비스는 공간 인지 및 데이터 관리의 혁신을 이끌고 있습니다. 이 블로그 포스트에서는 이 최신 기술의 동향, 주요 플레이어, 그리고 최신 뉴스를 분석하여 독자들이 이 분야에서 올바른 결정을 내릴 수 있도록 안내합니다.

**핵심 내용:**  

1. **온톨로지 기반 공간 AI 서비스란 무엇인가?**  
   - 【정의】온톨로지가 데이터와 정보 사이의 관계를 정의하는 지식 표현의 한 형태라는 점을 강조.
   - 【기술적 배경】AI와 결합된 온톨로지는 공간 데이터의 관리 및 분석에 혁신을 가져옴.

2. **최신 트렌드**  
   - 【개인화된 경험】온톨로지를 활용한 개인화된 공간 경험 제공.
   - 【데이터 통합】다양한 소스에서의 공간 데이터 통합 추세.
   - 【자동화과 제어】공간의 효율성 및 관리 최적화를 위한 자동화 솔루션 강조.

3. **주요 플레이어**  
   - 【기업들】온톨로지 기반 공간 AI 서비스에서 두각을 나타내는 기업들 목록 및 간략한 설명.
     - 예: Google AI, Microsoft Azure, IBM Watson 등.
   - 【스타트업】혁신을 주장하는 최신 스타트업의 소개 및 그들의 서비스 벤치마크.

4. **최신 뉴스 및 발전**  
   - 최근 뉴스 사건 및 각 기업의 새로운 제품 출시 정보.
   - 연구 결과나 업계 통계 

In [15]:
Markdown(result.raw)

# 온톨로지 기반 공간 AI 서비스의 최신 트렌드와 주요 플레이어

## 소개
온톨로지 기반 공간 AI 서비스는 우리가 공간을 인지하고 데이터를 관리하는 방식을 혁신적으로 변화시키고 있습니다. 이 블로그 포스트에서는 온톨로지와 AI가 결합한 최신 기술의 동향, 주요 플레이어, 그리고 최근 뉴스에 대해 심층 분석하여 독자들이 이 분야에서 올바른 결정을 내릴 수 있도록 돕고자 합니다. 지속적인 기술 발전과 데이터 통합이 중요한 시대에, 이번 글이 독자 여러분에게 많은 도움이 되길 바랍니다.

## 온톨로지 기반 공간 AI 서비스란 무엇인가?
### 정의
온톨로지는 데이터와 정보 사이의 관계를 정의하는 지식 표현의 한 형태로, 공간 데이터를 효율적으로 관리하는 데 중요한 역할을 합니다. 온톨로지 기반 AI는 이러한 기술을 활용하여 공간 데이터를 더욱 체계적이고 통합적으로 접근할 수 있게 합니다.

### 기술적 배경
AI와 결합된 온톨로지는 공간 데이터의 분석 및 관리에 혁신을 제공합니다. 데이터를 단순히 저장하는 것이 아니라, 의미 있는 패턴과 관계를 이해하고 분석함으로써 더 나은 의사결정을 지원할 수 있는 솔루션을 만들어냅니다. 따라서 온톨로지 기반의 공간 AI 서비스는 사용자의 요구에 맞는 개인화된 경험을 제공하는 데 중점을 두고 있습니다.

## 최신 트렌드
### 개인화된 경험
온톨로지는 개인화된 공간 경험을 제공하는 데 큰 기여를 하고 있습니다. 예를 들어, 사용자의 취향과 필요에 맞춘 공간 디자인이나 서비스 추천이 가능해졌습니다. 이는 사용자 참여를 촉진하고 더 나은 사용자 경험으로 이어집니다.

### 데이터 통합
최근의 트렌드는 다양한 소스에서 공간 데이터를 통합하는 것입니다. 이는 단일 플랫폼 안에서 여러 정보를 수집하고 분석함으로써, 더욱 심층적인 인사이트를 제공할 수 있는 토대를 마련합니다. 이러한 데이터 통합 솔루션은 스마트 시티 기술의 발전에도 크게 기여하고 있습니다.

### 자동화와 제어
공간의 효율성과 관리 최적화를 위한 자동화 솔루션이 점점 더 중요해지고 있습니다. 공간 AI는 운영의 자동화를 통해 관리자의 부담을 줄이고, 효율성을 극대화합니다. 이를 통해 더욱 스마트한 환경을 구축할 수 있습니다.

## 주요 플레이어
### 기업들
온톨로지 기반 공간 AI 서비스 분야에서 돋보이는 기업들로는 Google AI, Microsoft Azure, IBM Watson 등이 있습니다. 이들 기업은 강력한 AI 기술과 온톨로지를 결합하여 다양한 혁신적인 솔루션을 제공합니다.

### 스타트업
최근 혁신적인 아이디어로 등장한 스타트업들도 눈여겨볼 만합니다. 이들은 기존의 틀을 깨고 새로운 시장을 창출하고 있으며, 많은 투자자들이 그들의 성장 가능성을 주목하고 있습니다.

## 최신 뉴스 및 발전
최근 몇 가지 중요한 뉴스가 있었습니다. 각 기업들은 새로운 제품 출시와 연구 결과를 발표하며 시장에서의 경쟁력을 강화하고 있습니다. 예를 들어, IBM Watson은 온톨로지를 활용한 데이터 관리에 대한 새로운 연구 결과를 공개하였으며, 이는 업계의 주목을 받고 있습니다.

## 미래 전망
온톨로지 기반 공간 AI 서비스의 미래는 더욱 밝을 것으로 예상됩니다. 기술의 발전과 함께, 기업들은 AI와 온톨로지를 활용하여 더욱 정교한 데이터 분석과 사용자 맞춤형 솔루션을 제공할 것입니다. 우리가 앞으로 경험하게 될 변화들은 이 분야의 발전에 따라 매우 다양한 가능성을 열어줄 것입니다.

## 결론
온톨로지 기반 공간 AI 서비스는 앞으로도 지속적으로 발전할 분야이며, 다양한 기업과 스타트업이 이 경쟁에 참여하고 있습니다. 최신 기술 트렌드와 혁신적인 솔루션을 잘 이해하는 것이 중요합니다. 이 블로그 포스팅을 통해 여러분은 온톨로지 기반 공간 AI 서비스의 변화를 놓치지 않고, 향후 중요한 결정에 도움을 받을 수 있습니다. 더 알아보려면 [가입 링크]를 클릭해 최신 정보를 받아보세요!